In [29]:
# The imports

from dotenv import load_dotenv, dotenv_values
from agents import Agent, Runner, trace, SQLiteSession, function_tool
from agents.mcp import MCPServerStdio
import asyncio
import time
import os

In [30]:
load_dotenv(override=True)

True

In [31]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [32]:

# Configure Playwright to run in non-headless mode
playwright_params = {
    "command": "npx",
    "args": [
        "@playwright/mcp@latest",
        "--browser", "chrome",
        "--extension",   # connect to real Chrome via extension
        "--shared-browser-context",
    ],
    "env": {
        "PLAYWRIGHT_MCP_EXTENSION_TOKEN": "CELelKTMrkrFrc4QoB_dybke36loSalmSoz69QHGiWM"
    }
}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

In [33]:
@function_tool
def get_credentials(site_name: str):
    """ Takes site name as input and gives username and passowrd credentials of the site"""
    creds = dotenv_values(".env")
    username_key = f"{site_name.upper()}_USERNAME"
    password_key = f"{site_name.upper()}_PASSWORD"

    if username_key in creds and password_key in creds:
        return creds[username_key], creds[password_key]
    else:
        raise ValueError(f"Credentials for {site_name} ")


In [34]:
@function_tool
def wait_for_user(message: str = "Solve CAPTCHA in the browser and press Enter to continue"):
    """Pause execution until the user confirms."""
    print("Hello")
    time.sleep(7)
    return "Resumed after user solved CAPTCHA"

In [ ]:
instructions = """
You are an autonomous browsing agent.

General Browsing Instructions:

- Use ONE browser tab/page for all navigation. Do not open multiple tabs.
- When navigating to websites, use the full URL including https:// (e.g., https://youtube.com)
- Wait for pages to fully load before taking actions.
- Always accept cookies and dismiss pop-ups (e.g., click "Accept", "Not Now") when prompted.
- Close any unnecessary modals, banners, or pop-ups that appear while browsing.

Login Instructions:
- Use the get_credentials tool to retrieve the correct username and password for the website.
- Enter credentials in their respective fields (do not input both in the same box).
- After filling out username and password click on login button (Button might be such as Submit, Log in, etc.).
- If captcha is present do not click on login button. Call the 'wait_for_user' tool.


Website Navigation & Search Instructions:
- Navigate the internet autonomously to complete the users instructions.
- If one website fails to provide the required content, try alternative approaches or websites until the task is completed.
- Follow the users instructions precisely while searching or interacting with the website content.
- Be specific with URLs - for YouTube, use https://www.youtube.com
"""

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
    
    agent = Agent(
        name="investigator", 
        instructions=instructions, 
        model="gpt-5-mini",
        mcp_servers=[mcp_server_browser],
        tools=[get_credentials, wait_for_user]
    )

    with trace("new_trace"):
        result = await Runner.run(agent, "Login into student VTOP. Go to course page under academics section. Then open Database Systems and download Revathi C K materials", max_turns=20)
        print(result.final_output)

    #await asyncio.sleep(1000)


Hello
Hello
Done — I logged into VTOP, opened the Course Page and downloaded the materials uploaded by Revathi C K for Database Systems.

What I did:
- Logged into your VTOP student account.
- Navigated to Academics → Course Page (Course Page - view).
- Selected the course "BCSE302L - Database Systems - TH".
- Downloaded the course materials uploaded by REVATHI C K on 16-09-2025:
  - "Indxing, Hashing and Optimization" (Document)
  - "File organizatrion" (Document)

Notes / next steps:
- The downloads were triggered from the two download buttons next to those items on the Course Materials table.
- Do you want me to:
  - Also download materials for the Database Systems Lab (BCSE302P)?
  - Check other pages of the course-materials table for additional files by Revathi C K?
  - Upload the downloaded files to Google Drive / Dropbox or provide them as attachments here?

Tell me which (if any) of the above you want and I’ll proceed.
